<a href="https://colab.research.google.com/github/ThanhNhu-20104049/Project/blob/main/PROJECT%20CUOI%20KI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from tkinter import *
import tkinter as tk
from tkinter import ttk 
from tkinter import messagebox
from tkinter.filedialog import *
import googletrans
from googletrans import Translator
import speech_recognition as sr
from gtts import gTTS
import os
import playsound
from PIL import Image, ImageTk
import pytesseract 
import cv2


root = tk.Tk()
root.title('Language Translator')
root.geometry('620x450')
root.resizable(FALSE,FALSE)

frame1 = Frame(root, width=620,height=450,relief=RIDGE,borderwidth=5,bg='WHITE')
frame1.place(x=0,y=0)
Label(root, text='Language Translator',font=('Helvetica 20 bold'), fg='green',bg='WHITE').pack(pady=10)



key = list(googletrans.LANGUAGES.keys())
value = list(googletrans.LANGUAGES.values())
recognizer=sr.Recognizer()


def img_to_text(img,key):
	pytesseract.pytesseract.tesseract_cmd = r'C:\\Program Files\\Tesseract-OCR\\tesseract.exe'
	text = pytesseract.image_to_string(img,lang=key)
	return text

def camera():
	camera=cv2.VideoCapture('http://192.168.1.28:8080/video')
	while True:
		_,image=camera.read()
		cv2.imshow('Text detection',image)
		if cv2.waitKey(1)& 0xFF==ord('s'):
			cv2.imwrite('test.png',image)
			camera.release()
			cv2.destroyAllWindows()
			try:
				for i in range(len(value)):	
					if value[i]==choose_in.get():
						img_raw = cv2.imread('test.png')
						roi = cv2.selectROI(img_raw)
						roi_cropped = img_raw[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
						cv2.destroyAllWindows()
						text = img_to_text(roi_cropped,key[i])
						text_entry1.insert('end', text)
						translate()
			except:
				pass	
		if cv2.waitKey(1)& 0xFF==ord('e'):
			camera.release()
			cv2.destroyAllWindows()


def picture():
	clear()
	global file
	pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'
	if choose_in.get()!='Auto':
		for i in range(len(value)):
			if value[i]==choose_in.get():
				file = askopenfilename(defaultextension = '.jpg',filetypes = [('All Files','*.*'),('JPG Files','*.jpg'),('PNG Files','*.png')])				
				img_raw = cv2.imread(file)
				roi = cv2.selectROI(img_raw)
				roi_cropped = img_raw[int(roi[1]):int(roi[1]+roi[3]), int(roi[0]):int(roi[0]+roi[2])]
				cv2.destroyAllWindows()
				text = img_to_text(roi_cropped,key[i])
				text_entry1.insert('end', text)
				translate()
	else:
		messagebox.showerror('Language Translator','Please select language')

	
def voice():
	clear()
	with sr.Microphone() as source:
		recognizer.adjust_for_ambient_noise(source)
		audio=recognizer.listen(source)
	if choose_in.get()!='Auto':
		for i in range(len(value)):
			if value[i]==choose_in.get():
				result=recognizer.recognize_google(audio,language=key[i])
				text_entry1.insert('end', result)
				translate()
	else:
		messagebox.showerror('Language Translator','Please select language')
	translate()

def audio():
	file = askopenfilename(defaultextension = '.mp3',filetypes = [('All Files','*.*'),('WAV Files','*.wav'),('MP3 Files','*.mp3')])	
	r = sr.Recognizer() 
	with sr.AudioFile(file) as source:
		audio_data = r.record(source, offset = 1, duration = 60)
		for i in range(len(value)):
			if value[i]==choose_in.get():
				text = r.recognize_google(audio_data,language=key[i])
				text_entry1.insert('end', text)
				translate()

def speak():
	Lang_in = text_entry1.get('1.0','end-1c') 
	translator = Translator()
	detection = translator.detect(Lang_in)
	tts = gTTS(text=Lang_in, lang=detection.lang, slow=False)
	tts.save("sound.mp3")
	playsound.playsound("sound.mp3")
	os.remove("sound.mp3")

def speak1():
	Lang_out = text_entry2.get('1.0','end-1c')
	translator = Translator()
	detection = translator.detect(Lang_out)
	tts = gTTS(text=Lang_out, lang=detection.lang, slow=False)
	tts.save("sound1.mp3")
	playsound.playsound("sound1.mp3")
	os.remove("sound1.mp3")

def translate():
	cl = choose_out.get()
	Lang_in = text_entry1.get('1.0','end-1c')
	if Lang_in=='':
		messagebox.showerror('Language Translator','Enter the next to translate')
	else:
		text_entry2.delete(1.0,'end')
		translator = Translator()
		if choose_in.get()=='Auto':
			detection = translator.detect(Lang_in)
			output = translator.translate(Lang_in, src= detection.lang ,dest=cl)
		else:
			output = translator.translate(Lang_in, src= choose_in.get() ,dest=cl)
		text_entry2.insert('end', output.text)

def clear():
	text_entry1.delete(1.0,'end')
	text_entry2.delete(1.0,'end')

def convert():
	a = choose_in.get()
	b = choose_out.get()
	for i in range(len(value)):
		if value[i]==b:
			choose_in.current(i+1)
		if value[i]==a:
			choose_out.current(i)
	text_entry1.delete(1.0,'end')
	text_entry1.insert('end', text_entry2.get('1.0','end-1c'))
	text_entry2.delete(1.0,'end')
	translate()
		
a = tk.StringVar()

choose_in = ttk.Combobox(frame1, width=27, textvariable=a, state='rodomly',font=('verdana',10,'bold'))
b = list(googletrans.LANGUAGES.values())
b.insert(0,'Auto')
choose_in['values'] = (b)

choose_in.place(x=10,y=60)
choose_in.current(0)

l = tk.StringVar()
choose_out = ttk.Combobox(frame1, width=27, textvariable=l, state='rodomly',font=('verdana',10,'bold'))
choose_out['values']=(value)

choose_out.place(x=330,y=60)
choose_out.current(101)

text_entry1 = Text(frame1, width=20,height=10, borderwidth=2, relief=RIDGE,font=('Verdana',15))
text_entry1.place(x=10,y=105)

text_entry2 = Text(frame1, width=20,height=10, borderwidth=2, relief=RIDGE,font=('Verdana',15))
text_entry2.place(x=330,y=105)


img_trans= Image.open('D:\HKI-2022.2023\AI\Project\Trans.jpg')
img_trans = img_trans.resize((40,40))
img_trans=ImageTk.PhotoImage(img_trans)
btn = Button(frame1, relief=RAISED, borderwidth=0,image=img_trans, font=('verdana',10,'bold'), bg='skyblue', fg='white',cursor='hand2',command=translate)
btn.place(x=282,y=220)

img_cl= Image.open('D:\HKI-2022.2023\AI\Project\clear.png')
img_cl = img_cl.resize((15,15))
img_cl=ImageTk.PhotoImage(img_cl)
btn1 = Button(frame1, relief=RAISED, borderwidth=0,image=img_cl, font=('verdana',10,'bold'), fg='white',cursor='hand2',command=clear)
btn1.place(x=262,y=87)

img_mic= Image.open('D:\HKI-2022.2023\AI\Project\mic.png')
img_mic = img_mic.resize((30,30))
img_mic=ImageTk.PhotoImage(img_mic)
Mic = Button(frame1, relief=RAISED, borderwidth=0,image=img_mic,font=('verdana',10,'bold'), fg='white',cursor='hand2',command=voice)
Mic.place(x=207,y=328)

img_loa= Image.open('D:\HKI-2022.2023\AI\Project\loa.jpg')
img_loa = img_loa.resize((30,30))
img_loa=ImageTk.PhotoImage(img_loa)
Loa = Button(frame1, relief=RAISED, borderwidth=0,image=img_loa, font=('verdana',10,'bold'), fg='white',cursor='hand2', command=speak)
Loa.place(x=242,y=328)


img_loa1= Image.open('D:\HKI-2022.2023\AI\Project\loa.jpg')
img_loa1 = img_loa1.resize((30,30))
img_loa1=ImageTk.PhotoImage(img_loa1)
Loa1 = Button(frame1, relief=RAISED, borderwidth=0,image=img_loa1, font=('verdana',10,'bold'), fg='white',cursor='hand2', command=speak1)
Loa1.place(x=562,y=328)

img_cam= Image.open('D:\HKI-2022.2023\AI\Project\cam.jpg')
img_cam = img_cam.resize((40,40))
img_cam=ImageTk.PhotoImage(img_cam)
Pic = Button(frame1, relief=RAISED, borderwidth=0,image=img_cam, font=('verdana',10,'bold'), bg='skyblue', fg='white',cursor='hand2',command=camera)
Pic.place(x=227,y=375)

img_pic= Image.open('D:\HKI-2022.2023\AI\Project\Pic.jpg')
img_pic = img_pic.resize((40,40))
img_pic=ImageTk.PhotoImage(img_pic)
Pic = Button(frame1, relief=RAISED, borderwidth=0,image=img_pic, font=('verdana',10,'bold'), bg='skyblue', fg='white',cursor='hand2',command=picture)
Pic.place(x=282,y=375)

img_convert= Image.open('D:\HKI-2022.2023\AI\Project\convert.png')
img_convert = img_convert.resize((40,40))
img_convert=ImageTk.PhotoImage(img_convert)
Convert = Button(frame1, relief=RAISED, borderwidth=0,image=img_convert, font=('verdana',10,'bold'), bg='skyblue', fg='white',cursor='hand2',command=convert)
Convert.place(x=282,y=175)

img_audio= Image.open('D:\HKI-2022.2023\AI\Project\Audio.jpg')
img_audio = img_audio.resize((40,40))
img_audio=ImageTk.PhotoImage(img_audio)
audio = Button(frame1, relief=RAISED, borderwidth=0,image=img_audio, font=('verdana',10,'bold'), bg='skyblue', fg='white',cursor='hand2',command=audio)
audio.place(x=337,y=375)


root.mainloop()